In [21]:
# заполнение пропусков в данных
# библиотеки
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [22]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

In [23]:
energy = energy[(energy["building_id"]==0)]

In [24]:
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "floor_count"], axis=1)
del buildings
del weather
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           8784 non-null   object 
 1   building_id         8784 non-null   int64  
 2   meter_reading       8784 non-null   float64
 3   primary_use         8784 non-null   object 
 4   square_feet         8784 non-null   int64  
 5   year_built          8784 non-null   float64
 6   air_temperature     8781 non-null   float64
 7   cloud_coverage      4954 non-null   float64
 8   dew_temperature     8781 non-null   float64
 9   precip_depth_1_hr   8783 non-null   float64
 10  sea_level_pressure  8699 non-null   float64
 11  wind_direction      8534 non-null   float64
 12  wind_speed          8784 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 892.2+ KB
None


In [25]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [26]:
energy = reduce_mem_usage(energy)

Потребление памяти меньше на 0.62 Мб (минус 71.1 %)


In [27]:
# интерполяция данных
energy["precip_depth_1_hr"] = energy["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    energy[col] = energy[col].interpolate(limit_direction='both',
                            kind='cubic')

In [29]:
# проверка качества интерполяции
pd.set_option("use_inf_as_na", True)
for col in interpolate_columns:
    print(col, "Inf+NaN:", energy[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


In [30]:
# разделение данных
energy_train, energy_test = train_test_split(energy[energy["meter_reading"] > 0], test_size=0.2)
print(energy_train.head())


               timestamp  building_id  meter_reading primary_use  square_feet  \
7552 2016-11-10 16:00:00            0        181.500   Education         7432   
5852 2016-08-31 20:00:00            0        236.125   Education         7432   
5398 2016-08-12 22:00:00            0        309.250   Education         7432   
7568 2016-11-11 08:00:00            0        192.500   Education         7432   
6635 2016-10-03 11:00:00            0        251.125   Education         7432   

      year_built  air_temperature  cloud_coverage  dew_temperature  \
7552      2008.0        24.406250        2.000000        12.203125   
5852      2008.0        28.906250        8.000000        23.906250   
5398      2008.0        28.906250        5.199219        22.203125   
7568      2008.0        13.898438        0.000000        12.203125   
6635      2008.0        24.406250        6.332031        23.296875   

      precip_depth_1_hr  sea_level_pressure  wind_direction  wind_speed  
7552              

In [31]:
# линейная регрессия 
regression_columns = ["meter_reading", "air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
energy_train_lr = pd.DataFrame(energy_train,
                                columns=regression_columns)
y = energy_train_lr["meter_reading"]
x = energy_train_lr.drop(labels=["meter_reading"], axis=1)
model = LinearRegression().fit(x,y)
print(model.coef_, model.intercept_)

[ 2.66388817  3.78570262 -2.66543769 -2.07257617  0.17258061 -0.96963924] 1101.6054891140543


In [33]:
# предсказание и оценка модели
def calculate_model (x):
    lr = np.sum([x[col] * model.coef_[i] for i, col in enumerate(regression_columns[1:])])
    lr += model.intercept_
    x["meter_reading_lr_q"] = (np.log(1 + x.meter_reading) - np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model, axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print("Качество линейной регрессии", energy_test_lr_rmsle)

Качество линейной регрессии 0.20973244345825146
